In [ ]:
! pip install -U langgraph langsmith
! pip install langchain_community
! pip install langchain_openai

In [2]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END ,state
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage,AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import os

In [ ]:
import os
from langchain_openai import ChatOpenAI  # Notez le changement ici

# Définition de la clé API
os.environ['OPENAI_API_KEY'] = "sk-prouNnPRcA"

# Création du modèle
llm = ChatOpenAI(
    model="gpt-4o-mini",  # Notez que c'est 'model' et non 'model_name' dans les versions récentes
    temperature=0
)

In [4]:
active_prompt_template = ChatPromptTemplate.from_messages([
    ("system", """
    Generate Active Prompting examples based on user questions.
    
    Active Prompting is a specialized technique within Few-Shot Chain-of-Thought (CoT) prompting that:
    1. Starts with training questions/exemplars
    2. Asks the LLM to solve these exemplars
    3. Calculates uncertainty (disagreement in responses)
    4. Identifies exemplars with highest uncertainty
    5. Uses human annotators to rewrite these high-uncertainty exemplars
    6. Iteratively improves prompting quality through this feedback loop
    
    This approach enhances LLM performance by focusing human effort on the most challenging examples.
    
    Please structure your response in the following format:
    
    Step 1: Initial Exemplars
    [Provide 3-5 exemplars with questions and CoT reasoning]
    
    Step 2: First Iteration
    [LLM-generated solutions to the exemplars]
    
    Step 3: Uncertainty Analysis
    [Identify which exemplars have highest uncertainty/disagreement]
    
    Step 4: Refined Exemplars
    [Rewritten versions of high-uncertainty exemplars with clearer reasoning chains]
    
    Step 5: Final Solution
    [Improved answer to the original question using insights from the refined exemplars]
    
    Example:
    Question: "What would be the total cost of buying 3 shirts at $24.99 each and 2 pairs of pants at $39.95 each, with an 8% sales tax?"
    
    Initial Exemplars:
    Exemplar 1: 
    Q: If a book costs $12.50 and I have a 20% off coupon, how much will I pay?
    A: To find the discounted price, I need to calculate 20% of $12.50 and subtract it from the original price.
    20% of $12.50 = 0.20 × $12.50 = $2.50
    Final price = $12.50 - $2.50 = $10.00
    Therefore, I will pay $10.00 for the book.
    
    Exemplar 2:
    Q: What is the total cost of 4 apples at $0.75 each and 3 oranges at $0.60 each?
    A: First, I'll calculate the cost of the apples.
    Cost of apples = 4 × $0.75 = $3.00
    Next, I'll calculate the cost of the oranges.
    Cost of oranges = 3 × $0.60 = $1.80
    Total cost = $3.00 + $1.80 = $4.80
    Therefore, the total cost is $4.80.
    
    Exemplar 3:
    Q: If a meal costs $45.00 and I want to leave a 15% tip, what will my total bill be?
    A: To find the tip amount, I calculate 15% of the meal cost.
    Tip amount = 0.15 × $45.00 = $6.75
    Total bill = Meal cost + Tip = $45.00 + $6.75 = $51.75
    Therefore, my total bill will be $51.75.
    
    First Iteration:
    For our question about shirts and pants:
    Cost of shirts = 3 × $24.99 = $74.97
    Cost of pants = 2 × $39.95 = $79.90
    Subtotal = $74.97 + $79.90 = $154.87
    Tax amount = 0.08 × $154.87 = $12.39
    Total cost = $154.87 + $12.39 = $167.26
    
    Uncertainty Analysis:
    The calculation seems straightforward, but there might be uncertainty in how to handle the cents in the multiplication and whether to round the tax amount. 
     Let's identify this as an area of potential disagreement.
    
    Refined Exemplars:
    Exemplar 2 (Rewritten):
    Q: What is the total cost of 4 apples at $0.75 each and 3 oranges at $0.60 each?
    A: First, I'll calculate the cost of the apples.
    Cost of apples = 4 × $0.75 = $3.00
    Next, I'll calculate the cost of the oranges.
    Cost of oranges = 3 × $0.60 = $1.80
    Subtotal = $3.00 + $1.80 = $4.80
    For retail transactions, we calculate to the nearest cent, so the total cost is exactly $4.80.
    
    Exemplar 3 (Rewritten):
    Q: If a meal costs $45.00 and I want to leave a 15% tip, what will my total bill be?
    A: To find the tip amount, I calculate 15% of the meal cost.
    Tip amount = 0.15 × $45.00 = $6.75
    Total bill = Meal cost + Tip = $45.00 + $6.75 = $51.75
    When calculating percentages, we typically round to the nearest cent, so the total bill is $51.75.
    
    Final Solution:
    To find the total cost of buying 3 shirts and 2 pairs of pants with sales tax:
    
    Step 1: Calculate the cost of the shirts.
    Cost of shirts = 3 × $24.99 = $74.97
    
    Step 2: Calculate the cost of the pants.
    Cost of pants = 2 × $39.95 = $79.90
    
    Step 3: Determine the subtotal before tax.
    Subtotal = $74.97 + $79.90 = $154.87
    
    Step 4: Calculate the sales tax amount.
    Tax amount = 0.08 × $154.87 = $12.3896
    Rounded to the nearest cent: $12.39
    
    Step 5: Compute the final total.
    Total cost = $154.87 + $12.39 = $167.26
    
    Therefore, the total cost would be $167.26.
    """),
    ("human", "{question}"),
    ("assistant", """
    I'll solve this problem using Active Prompting to iteratively improve my approach:
    
    Step 1: Initial Exemplars
    [Provide 3 relevant exemplars with detailed reasoning paths that relate to the problem domain]
    
    Step 2: First Iteration
    [Initial detailed solution attempt with step-by-step reasoning]
    
    Step 3: Uncertainty Analysis
    [Identification of specific steps or concepts where reasoning might be unclear or where multiple approaches could lead to different answers]
    
    Step 4: Refined Exemplars
    [Improved versions of the exemplars that address the identified uncertainties with clearer reasoning chains]
    
    Step 5: Final Solution
    [Comprehensive step-by-step solution that leverages insights from the refined exemplars and addresses all aspects of the original question]
    """)
])

In [7]:
import pandas as pd
import re
from typing import Optional, TypedDict, Annotated
from langchain.schema import AIMessage, HumanMessage

# Function to generate the Contrastive Chain of Thought prompt and extract the final answer (Step 5)
def generate_active_prompt_node(state):
    question = state['messages'][-1].content  # Get the last question
    prompt_value = active_prompt_template.invoke({"question": question})
    messages = prompt_value.to_messages()
    response = llm.invoke(messages)
    
    # Keep the full response for processing
    full_response = response.content
    
    # Extract the final answer from Step 5
    final_answer_pattern = r'Step 5:([\s\S]*?)(?=$|Step \d:)'
    final_answer_match = re.search(final_answer_pattern, full_response)
    
    if final_answer_match:
        final_solution = final_answer_match.group(1).strip()
    else:
        # Alternative pattern
        alternative_pattern = r'Step 5: \[Final answer with justification\]([\s\S]*?)(?=$|Step \d:)'
        alternative_match = re.search(alternative_pattern, full_response)
        
        if alternative_match:
            final_solution = alternative_match.group(1).strip()
        else:
            final_solution = "Final solution not found in the expected format."
    
    # Save both the question, final answer, and full processing to a text file
    with open('active_prompt_result.txt', 'w', encoding='utf-8') as f:
        f.write("=== QUESTION ===\n\n")
        f.write(question)
        f.write("\n\n=== FINAL ANSWER (STEP 5) ===\n\n")
        f.write(final_solution)
        f.write("\n\n=== COMPLETE PROCESSING (FOR REFERENCE) ===\n\n")
        f.write(full_response)
    
    # Return only the final answer in messages, plus the original question for reference
    return {
        "messages": [AIMessage(content=final_solution)],
        "question": question,
        "full_response": full_response
    }

# Definition of state
class State(TypedDict):
    messages: Annotated[list, add_messages]
    question: Optional[str]
    full_response: Optional[str]

# Graph creation
graph_builder = StateGraph(State)
graph_builder.add_node("generate_active_prompt_cot", generate_active_prompt_node)

# Graph configuration
graph_builder.set_entry_point("generate_active_prompt_cot")
graph_builder.add_edge("generate_active_prompt_cot", END)
graph = graph_builder.compile()

# Usage example
inputs = {"messages": [HumanMessage(content="Marie gave half of her apples to Jean. Jean ate three apples," 
                                         " then gave half of what he had left to Pierre. " 
                                         "Pierre now has 4 apples. How many apples did Marie have at the beginning?")],
          "result_df": None}  # Initialize result_df to None


# Print the question at the beginning
print("QUESTION:")
print(inputs["messages"][0].content)
print("\nFINAL ANSWER (STEP 5):")

# Graph execution - only display Step 5
original_question = ""
for output in graph.stream(inputs):
    for key, value in output.items():
        if key == "generate_active_prompt_cot":
            messages = value['messages']
            for message in messages:
                if isinstance(message, AIMessage):
                    print(message.content)
            
            # Store question for reference
            if "question" in value:
                original_question = value["question"]

# Print confirmation that results were saved
print("\n(Complete results saved in 'active_prompt_result.txt')")

QUESTION:
Marie gave half of her apples to Jean. Jean ate three apples, then gave half of what he had left to Pierre. Pierre now has 4 apples. How many apples did Marie have at the beginning?

FINAL ANSWER (STEP 5):
Final Solution**

To find out how many apples Marie had initially, we can follow these steps:

1. Let M be the number of apples Marie had initially.
2. Marie gave half of her apples to Jean: Jean received M/2 apples.
3. Jean ate 3 apples: Now he has (M/2 - 3) apples left.
4. Jean then gave half of what he had left to Pierre: Pierre received (M/2 - 3)/2 apples.
5. We know Pierre has 4 apples, so we set up the equation:  
   (M/2 - 3)/2 = 4.
6. Solving for M:  
   Multiply both sides by 2: M/2 - 3 = 8.  
   Add 3 to both sides: M/2 = 11.  
   Multiply both sides by 2: M = 22.

Therefore, Marie had 22 apples at the beginning.

(Complete results saved in 'active_prompt_result.txt')
